# Import Required Libraries

In [289]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import configparser

# Create Random Datasets for Creating DataFrame

In [19]:

# Define the number of months and the initial retention rate
num_months = 10000
initial_retention_rate = 0.2  # Initial monthly retention rate

# Function to generate realistic monthly retention rates
def generate_retention_rates(num_months, initial_rate):
    months = list(range(1, num_months + 1))
    retention_rates = []
    for month in months:
        # Model logistic growth to gradually increase retention rates over time
        retention_rate = 1 / (1 + np.exp(-0.5 * (month - 6)))
        # Scale the retention rate based on the initial rate
        retention_rate = initial_rate + (1 - initial_rate) * retention_rate
        retention_rates.append(retention_rate)
    return retention_rates

# Generate realistic monthly retention rates
retention_rates = generate_retention_rates(num_months, initial_retention_rate)

# Generate random data for each column
data = {
    'record_date': [datetime.now().date() - timedelta(days=30*i) for i in range(num_months)],
    'beginning_users': [random.randint(1000, 5000) for _ in range(num_months)],
    'new_users': [random.randint(50, 500) for _ in range(num_months)],
    'lost_users': [random.randint(20, 200) for _ in range(num_months)],
    'logins': [random.randint(100, 1000) for _ in range(num_months)]
}

# Calculate end users
data['end_users'] = [beginning + new - lost for beginning, new, lost in zip(data['beginning_users'], data['new_users'], data['lost_users'])]

# Calculate active users using retention rates
data['active_users'] = [int(end_users * retention_rate) for end_users, retention_rate in zip(data['end_users'], retention_rates)]
data['monthly_retention'] = retention_rates
data['monthly_churn'] = 1 - np.array(retention_rates)

# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
display(df.head())

,record_date,beginning_users,new_users,lost_users,logins,end_users,active_users,monthly_retention,monthly_churn
0,2024-03-31,2653,443,194,385,2902,756,0.260687,0.739313
1,2024-03-01,2571,155,177,794,2549,752,0.295362,0.704638
2,2024-01-31,1077,380,106,379,1351,467,0.345940,0.654060
3,2024-01-01,3925,302,198,154,4029,1672,0.415153,0.584847
4,2023-12-02,4981,349,20,732,5310,2665,0.502033,0.497967


# Load Random Data to Postgresql Database

In [294]:
#Assign required credentials from con.txt file
config = configparser.ConfigParser()
config.read('.env')

#INPUT YOUR OWN CONNECTION STRING HERE
conn_string = config['DBCRED']['conn']

#perform to_sql test and print result
db = create_engine(conn_string)
conn = db.connect()
print(f"writing {len(df)} records")
df.to_sql('user_activity', con=conn, if_exists='replace', index=False)

OperationalError: (psycopg2.OperationalError) could not translate host name "hy-054d67e6-2541-4026-9863-215318564602.us-east-1.aws.hydradb.io" to address: No such host is known. 

(Background on this error at: https://sqlalche.me/e/14/e3q8)

# Data Cleaning

In [191]:
df_gps = pd.read_csv("googleplaystore.csv")

In [192]:
df_gps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [193]:
df_gps = df_gps[df_gps["Category"]!="1.9"]

In [194]:
df_gps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10840 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10840 non-null  object 
 1   Category        10840 non-null  object 
 2   Rating          9366 non-null   float64
 3   Reviews         10840 non-null  object 
 4   Size            10840 non-null  object 
 5   Installs        10840 non-null  object 
 6   Type            10839 non-null  object 
 7   Price           10840 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10840 non-null  object 
 11  Current Ver     10832 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.2+ MB


In [195]:
df_gps.dropna(inplace=True)

In [196]:
df_gps.isnull().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
dtype: int64

In [197]:
df_gps.drop_duplicates(inplace=True)

In [198]:
df_gps.Category = df_gps.Category.astype("string")

In [199]:
df_gps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8886 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             8886 non-null   object 
 1   Category        8886 non-null   string 
 2   Rating          8886 non-null   float64
 3   Reviews         8886 non-null   object 
 4   Size            8886 non-null   object 
 5   Installs        8886 non-null   object 
 6   Type            8886 non-null   object 
 7   Price           8886 non-null   object 
 8   Content Rating  8886 non-null   object 
 9   Genres          8886 non-null   object 
 10  Last Updated    8886 non-null   object 
 11  Current Ver     8886 non-null   object 
 12  Android Ver     8886 non-null   object 
dtypes: float64(1), object(11), string(1)
memory usage: 971.9+ KB


In [200]:
df_gps.App = df_gps.App.astype("string")

In [201]:
df_gps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [202]:
df_gps["Reviews"] = df_gps["Reviews"].astype("int")

In [203]:
df_gps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8886 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             8886 non-null   string 
 1   Category        8886 non-null   string 
 2   Rating          8886 non-null   float64
 3   Reviews         8886 non-null   int32  
 4   Size            8886 non-null   object 
 5   Installs        8886 non-null   object 
 6   Type            8886 non-null   object 
 7   Price           8886 non-null   object 
 8   Content Rating  8886 non-null   object 
 9   Genres          8886 non-null   object 
 10  Last Updated    8886 non-null   object 
 11  Current Ver     8886 non-null   object 
 12  Android Ver     8886 non-null   object 
dtypes: float64(1), int32(1), object(9), string(2)
memory usage: 937.2+ KB


In [204]:
df_gps = df_gps[df_gps["Size"]!="Varies with device"]

In [205]:
def k_size(value):
    if "M" in value:
        return int(float(value.split("M")[0])*1000)
    elif "k" in value:
        return int(float(value.split("k")[0]))
df_gps["Size"] = df_gps["Size"].apply(k_size)


In [206]:
df_gps["Size"] = df_gps["Size"].apply(lambda x : x/1000)

In [207]:
df_gps[df_gps["Size"]==0]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver


In [213]:
df_gps.rename(columns = {"Size":"Size(MB)"},inplace=True)

In [214]:
df_gps.reset_index(inplace=True)
del df_gps["index"]

In [215]:
df_gps

,App,Category,Rating,Reviews,Size(MB),Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.000,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.000,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.700,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.000,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.800,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7413,Chemin (fr),BOOKS_AND_REFERENCE,4.8,44,0.619,"1,000+",Free,0,Everyone,Books & Reference,"March 23, 2014",0.8,2.2 and up
7414,FR Calculator,FAMILY,4.0,7,2.600,500+,Free,0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up
7415,Sya9a Maroc - FR,FAMILY,4.5,38,53.000,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
7416,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.600,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up


In [216]:
df_gps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7418 entries, 0 to 7417
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             7418 non-null   string 
 1   Category        7418 non-null   string 
 2   Rating          7418 non-null   float64
 3   Reviews         7418 non-null   int32  
 4   Size(MB)        7418 non-null   float64
 5   Installs        7418 non-null   object 
 6   Type            7418 non-null   object 
 7   Price           7418 non-null   object 
 8   Content Rating  7418 non-null   object 
 9   Genres          7418 non-null   object 
 10  Last Updated    7418 non-null   object 
 11  Current Ver     7418 non-null   object 
 12  Android Ver     7418 non-null   object 
dtypes: float64(2), int32(1), object(8), string(2)
memory usage: 724.5+ KB


In [221]:
df_gps["Installs"] = df_gps["Installs"].apply(lambda x : x.split("+")[0])

In [227]:
df_gps["Installs"] = df_gps["Installs"].apply(lambda x : "".join(x.split(","))).astype("int")

In [237]:
df_gps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7418 entries, 0 to 7417
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             7418 non-null   string 
 1   Category        7418 non-null   string 
 2   Rating          7418 non-null   float64
 3   Reviews         7418 non-null   int32  
 4   Size(MB)        7418 non-null   float64
 5   Installs        7418 non-null   int32  
 6   Type            7418 non-null   string 
 7   Price           7418 non-null   object 
 8   Content Rating  7418 non-null   object 
 9   Genres          7418 non-null   object 
 10  Last Updated    7418 non-null   object 
 11  Current Ver     7418 non-null   object 
 12  Android Ver     7418 non-null   object 
dtypes: float64(2), int32(2), object(6), string(3)
memory usage: 695.6+ KB


In [236]:
df_gps["Type"] = df_gps["Type"].astype("string")

In [249]:
df_gps["Price"].unique()

array(['0', '$4.99', '$6.99', '$7.99', '$3.99', '$5.99', '$2.99', '$1.99',
       '$9.99', '$0.99', '$9.00', '$5.49', '$10.00', '$24.99', '$11.99',
       '$79.99', '$16.99', '$14.99', '$29.99', '$12.99', '$3.49',
       '$10.99', '$7.49', '$1.50', '$19.99', '$15.99', '$33.99', '$39.99',
       '$2.49', '$4.49', '$1.70', '$1.49', '$3.88', '$399.99', '$17.99',
       '$400.00', '$3.02', '$1.76', '$4.84', '$4.77', '$1.61', '$1.59',
       '$6.49', '$1.29', '$299.99', '$379.99', '$37.99', '$18.99',
       '$389.99', '$8.49', '$1.75', '$14.00', '$2.00', '$3.08', '$2.59',
       '$19.40', '$15.46', '$8.99', '$3.04', '$13.99', '$4.29', '$3.28',
       '$4.60', '$1.00', '$2.90', '$1.97', '$2.56', '$1.20'], dtype=object)

In [253]:
df_gps["Price"] = df_gps["Price"].apply(lambda x : float(x.split("$")[1]) if "$" in x else float(x))

In [282]:
df_gps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7418 entries, 0 to 7417
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   App             7418 non-null   string        
 1   Category        7418 non-null   string        
 2   Rating          7418 non-null   float32       
 3   Reviews         7418 non-null   int32         
 4   Size(MB)        7418 non-null   float32       
 5   Installs        7418 non-null   int32         
 6   Type            7418 non-null   string        
 7   Price($)        7418 non-null   float32       
 8   Content Rating  7418 non-null   string        
 9   Genres          7418 non-null   string        
 10  Last Updated    7418 non-null   datetime64[ns]
 11  Current Ver     7418 non-null   object        
 12  Android Ver     7418 non-null   object        
dtypes: datetime64[ns](1), float32(3), int32(2), object(2), string(5)
memory usage: 608.6+ KB


In [257]:
df_gps["Rating"] = df_gps["Rating"].astype("float32")

In [258]:
df_gps["Size(MB)"] = df_gps["Size(MB)"].astype("float32")

In [259]:
df_gps["Price"] = df_gps["Price"].astype("float32")

In [267]:
df_gps.rename(columns =  {"Price":"Price($)"},inplace=True)

In [272]:
df_gps["Content Rating"] = df_gps["Content Rating"].astype("string")

In [277]:
df_gps["Genres"] = df_gps["Genres"].astype("string")

In [281]:
df_gps["Last Updated"] = pd.to_datetime(df_gps["Last Updated"])

In [302]:
df_gps.head()

,App,Category,Rating,Reviews,Size(MB),Installs,Type,Price($),Content Rating,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,2018-01-07,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,2018-01-15,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,2018-08-01,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,2018-06-08,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,2018-06-20,1.1,4.4 and up


In [303]:
#Assign required credentials from con.txt file
config = configparser.ConfigParser()
config.read('.env')

#INPUT YOUR OWN CONNECTION STRING HERE
conn_string = config['DBCRED']['conn']

#perform to_sql test and print result
db = create_engine(conn_string)
conn = db.connect()
print(f"writing {len(df_gps)} records")
df_gps.to_sql('google_play_service', con=conn, if_exists='replace', index=False)

writing 7418 records


418

In [301]:
del df_gps["Genres"]

In [306]:
df_gps["Size(MB)"].unique()

array([1.90e+01, 1.40e+01, 8.70e+00, 2.50e+01, 2.80e+00, 5.60e+00,
       2.90e+01, 3.30e+01, 3.10e+00, 2.80e+01, 1.20e+01, 2.00e+01,
       2.10e+01, 3.70e+01, 5.50e+00, 1.70e+01, 3.90e+01, 3.10e+01,
       4.20e+00, 2.30e+01, 6.00e+00, 6.10e+00, 4.60e+00, 9.20e+00,
       5.20e+00, 1.10e+01, 2.40e+01, 9.40e+00, 1.50e+01, 1.00e+01,
       1.20e+00, 2.60e+01, 8.00e+00, 7.90e+00, 5.60e+01, 5.70e+01,
       3.50e+01, 5.40e+01, 2.01e-01, 3.60e+00, 5.70e+00, 8.60e+00,
       2.40e+00, 2.70e+01, 2.70e+00, 2.50e+00, 7.00e+00, 1.60e+01,
       3.40e+00, 8.90e+00, 3.90e+00, 2.90e+00, 3.80e+01, 3.20e+01,
       5.40e+00, 1.80e+01, 1.10e+00, 2.20e+00, 4.50e+00, 9.80e+00,
       5.20e+01, 9.00e+00, 6.70e+00, 3.00e+01, 2.60e+00, 7.10e+00,
       2.20e+01, 6.40e+00, 3.20e+00, 8.20e+00, 4.90e+00, 9.50e+00,
       5.00e+00, 5.90e+00, 1.30e+01, 7.30e+01, 6.80e+00, 3.50e+00,
       4.00e+00, 2.30e+00, 2.10e+00, 4.20e+01, 9.10e+00, 5.50e+01,
       2.30e-02, 7.30e+00, 6.50e+00, 1.50e+00, 7.50e+00, 5.10e